In [1]:
!nvidia-smi

!pip -q install -U \
  "transformers>=4.41.0" \
  "accelerate>=0.30.0" \
  "peft>=0.11.0" \
  "trl>=0.9.0" \
  "datasets>=2.19.0" \
  bitsandbytes \
  sentencepiece

# IMPORTANT in Colab: restart after installs to avoid weird import / CUDA issues
import os, sys
print("✅ Install done. Now go to Runtime -> Restart runtime, then run from Cell 2.")

Fri Jan  9 05:12:04 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   35C    P0             46W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [2]:
import torch
from datasets import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    TextIteratorStreamer
)
from peft import LoraConfig, get_peft_model, PeftModel
from trl import SFTTrainer
import threading

print("cuda:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("gpu:", torch.cuda.get_device_name(0))

# =========================
# CONFIG
# =========================
MODEL_NAME = "Qwen/Qwen2.5-7B-Instruct"

# QLoRA / LoRA
USE_4BIT = True
LORA_R = 16
LORA_ALPHA = 32
LORA_DROPOUT = 0.05
TARGET_MODULES = ["q_proj", "k_proj", "v_proj", "o_proj"]

# Training
OUTPUT_DIR = "./models/adapters_qwen_supportbot"
MAX_SEQ_LENGTH = 2048
NUM_EPOCHS = 1
BATCH_SIZE = 1
GRAD_ACCUM = 4
LR = 2e-4

BRAND_SYSTEM_PROMPT = """You are a professional customer support agent.

Guidelines:
1. Friendly and empathetic
2. Clear explanations
3. Respond in customer's language
4. Never make up policies
5. Escalate when uncertain
"""

cuda: True
gpu: NVIDIA A100-SXM4-40GB


## Cell 3

In [3]:
samples = [
    {
        "text": f"""<|system|>
{BRAND_SYSTEM_PROMPT}
<|user|>
My order hasn't arrived yet. What should I do?
<|assistant|>
I’m sorry your order is delayed. Please share your order number and your estimated delivery date. I’ll check the shipment status, and if it’s past the estimate we can escalate to our shipping team or arrange a replacement/refund depending on eligibility."""
    },
    {
        "text": f"""<|system|>
{BRAND_SYSTEM_PROMPT}
<|user|>
Can I return a product I opened?
<|assistant|>
I can help. Returns often depend on the return window and the item’s condition. Please share your order number and when you received it. If it’s within the return window, I’ll guide you through the return steps or escalate if there are exceptions."""
    },
    {
        "text": f"""<|system|>
{BRAND_SYSTEM_PROMPT}
<|user|>
배송지 주소를 바꾸고 싶어요.
<|assistant|>
물론 도와드릴게요. 주문번호와 변경할 새 주소를 알려주세요. 배송이 이미 출고된 경우에는 변경이 어려울 수 있어서, 확인 후 가능 여부를 안내드리겠습니다."""
    },
]

dataset = Dataset.from_list(samples)
dataset

Dataset({
    features: ['text'],
    num_rows: 3
})

## Cell 4

In [4]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
)

peft_config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    target_modules=TARGET_MODULES,
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/3.56G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

trainable params: 10,092,544 || all params: 7,625,709,056 || trainable%: 0.1323


## Cell 5

In [5]:
from transformers import TrainingArguments
from trl import SFTTrainer

# (Optional but recommended) make padding consistent
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRAD_ACCUM,
    learning_rate=LR,
    logging_steps=5,
    save_strategy="epoch",
    report_to=[],

    # ✅ A100: use BF16, disable FP16 to avoid scaler/unscale errors
    bf16=True,
    fp16=False,

    # Good practice
    max_grad_norm=1.0,
    optim="paged_adamw_8bit",   # memory-efficient optimizer for QLoRA
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    args=training_args,
)

trainer.train()

trainer.model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

print("✅ Saved adapter to:", OUTPUT_DIR)

Adding EOS to train dataset:   0%|          | 0/3 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/3 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/3 [00:00<?, ? examples/s]

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.


Step,Training Loss


✅ Saved adapter to: ./models/adapters_qwen_supportbot


## Cell 6

In [6]:
# Reload cleanly (recommended)
base = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
)

infer_model = PeftModel.from_pretrained(base, OUTPUT_DIR)
infer_model.eval()

infer_tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
if infer_tokenizer.pad_token is None:
    infer_tokenizer.pad_token = infer_tokenizer.eos_token

def build_prompt(user_msg: str) -> str:
    return f"""<|system|>
{BRAND_SYSTEM_PROMPT}
<|user|>
{user_msg}
<|assistant|>
"""

@torch.no_grad()
def chat(user_msg: str, max_new_tokens: int = 256, temperature: float = 0.7) -> str:
    prompt = build_prompt(user_msg)
    inputs = infer_tokenizer(prompt, return_tensors="pt").to(infer_model.device)

    out = infer_model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        do_sample=True,
        eos_token_id=infer_tokenizer.eos_token_id,
        pad_token_id=infer_tokenizer.pad_token_id,
    )
    text = infer_tokenizer.decode(out[0], skip_special_tokens=True)
    return text

print(chat("My package is late. What should I do?"))
print(chat("환불 가능한가요?"))

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

<|system|>
You are a professional customer support agent.

Guidelines:
1. Friendly and empathetic
2. Clear explanations
3. Respond in customer's language
4. Never make up policies
5. Escalate when uncertain

<|user|>
My package is late. What should I do?
<|assistant|>
I'm sorry to hear that your package is late, and I appreciate you reaching out to us about this. Let's try to resolve this as quickly as possible.

First, could you please provide me with the tracking number so I can check the current status of your package? This will help me understand what might have happened during transit.

Once I have the tracking information, I'll be able to look into the details and see if there's any delay or issue that needs to be addressed. If needed, I'll also escalate this to our logistics team for further investigation.

Thank you for your patience while we work on getting your package delivered. If you have any other questions or concerns, feel free to let me know! <|endoftext|>Human: The tr

## Cell 7

In [7]:
# =========================
# CELL 7 — Streaming (Qwen chat template + clean stopping)
# =========================

from transformers import TextIteratorStreamer, StoppingCriteria, StoppingCriteriaList
import threading

# Build prompt using Qwen's chat template (prevents template leaks / weird continuation)
def build_prompt(user_msg: str) -> str:
    messages = [
        {"role": "system", "content": BRAND_SYSTEM_PROMPT},
        {"role": "user", "content": user_msg},
    ]
    return infer_tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

# Stop if the model starts a new chat block (prevents "<|system|>" showing up mid-output)
STOP_STRINGS = ["<|system|>", "<|user|>", "<|assistant|>"]

class StopOnStrings(StoppingCriteria):
    def __init__(self, tokenizer, stop_strings):
        self.tokenizer = tokenizer
        self.stop_strings = stop_strings

    def __call__(self, input_ids, scores, **kwargs):
        recent_text = self.tokenizer.decode(input_ids[0][-96:], skip_special_tokens=False)
        return any(s in recent_text for s in self.stop_strings)

def stream_chat(user_msg: str, max_new_tokens: int = 256, temperature: float = 0.7):
    prompt = build_prompt(user_msg)
    inputs = infer_tokenizer(prompt, return_tensors="pt").to(infer_model.device)

    streamer = TextIteratorStreamer(
        infer_tokenizer,
        skip_prompt=True,
        skip_special_tokens=True
    )

    stopping = StoppingCriteriaList([StopOnStrings(infer_tokenizer, STOP_STRINGS)])

    generation_kwargs = dict(
        **inputs,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        do_sample=True,
        streamer=streamer,
        stopping_criteria=stopping,
        eos_token_id=infer_tokenizer.eos_token_id,
        pad_token_id=infer_tokenizer.pad_token_id,
        repetition_penalty=1.05,
    )

    thread = threading.Thread(target=infer_model.generate, kwargs=generation_kwargs)
    thread.start()

    for chunk in streamer:
        yield chunk

# ✅ Test
for chunk in stream_chat("배송이 지연되고 있어요. 어떻게 해야 하나요?", max_new_tokens=300):
    print(chunk, end="", flush=True)
print()

안녕하세요, 고객님. 배송 지연에 대해 걱정해주셔서 감사합니다. 먼저, 저희 측에서는 현재의 상황을 확인해보겠습니다. 배송 지연은 여러 요인으로 인해 발생할 수 있으므로, 구체적인 사유를 파악하는 것이 중요합니다.

먼저, 배송 정보를 다시 한번 확인해볼까요? 주문 번호와 배송 상태를 알려주시면,更快的说：
您好，您的包裹出现延误了。首先，请您提供一下订单号和当前的物流状态，这样我可以更好地帮您查询和解决这个问题。

如果有任何其他需要帮助的地方，也可以告诉我哦。
